In [6]:
import pandas as pd
from datetime import datetime, timedelta

def read_and_process_data(file_path):
    # Load data, assuming it's separated by spaces
    data = pd.read_csv('Idena.txt', delimiter=' ', names=['Token', 'Price', 'DateTime'], parse_dates=['DateTime'])
    
    # Convert timestamp to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    
    # Sort by timestamp just in case
    data.sort_values('timestamp', inplace=True)
    
    return data

def analyze_price_changes(data, analysis_period=timedelta(hours=1), interval=timedelta(minutes=15), threshold=0.10):
    end_time = data['timestamp'].max()
    start_time = end_time - analysis_period
    significant_changes = []

    current_time = start_time
    while current_time < end_time:
        interval_start = current_time
        interval_end = min(current_time + interval, end_time)
        interval_data = data[(data['timestamp'] >= interval_start) & (data['timestamp'] <= interval_end)]
        
        if not interval_data.empty:
            price_change = (interval_data['price'].iloc[-1] - interval_data['price'].iloc[0]) / interval_data['price'].iloc[0]
            if abs(price_change) >= threshold:
                significant_changes.append({
                    'start': interval_start,
                    'end': interval_end,
                    'price_change': price_change
                })
        
        current_time += interval
    
    return significant_changes

def main():
    file_path = 'Idena.txt'
    data = read_and_process_data(file_path)
    significant_changes = analyze_price_changes(data)
    
    # Generate report or send notifications based on significant_changes
    for change in significant_changes:
        print(f"Significant change from {change['start']} to {change['end']}: {change['price_change']:.2%}")

if __name__ == "__main__":
    main()


KeyError: 'timestamp'